In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torchvision
import random
import os
import argparse
import numpy as np
import importlib
from cnn import Net
import torch.optim as optim
from torchvision import datasets, transforms
import time
import Helper
import regular
import interpretable
from Helper import save_checkpoint

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('./data/MNIST_data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('./data/MNIST_data', train=False,
                    transform=transform)
trainloader = torch.utils.data.DataLoader(dataset1,batch_size=512, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset2, batch_size=1, shuffle=False)

In [ ]:
model = Net()
#model.load_state_dict(torch.load('models/MNIST_regular1_model.th')['state_dict'])
#model.load_state_dict(torch.load('models/MNIST_interpretablefeaturesDroped_0.5_RandomMasking_1_model.th')['state_dict'])
model.load_state_dict(torch.load('models/MNIST_interpretablefeaturesDroped_0.3_RandomMasking_1_model.th')['state_dict'])
model.eval()

In [ ]:
test_loss = 0
correct = 0
model.cuda()
with torch.no_grad():
    for data, target in testloader:
        output = model(data.cuda())
        pred = output.argmax(dim=1, keepdim=True)
        #print(pred)
        correct +=  torch.eq(pred, target.cuda()).sum().item()
test_loss /= len(testloader.dataset)
print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.format(
     correct, len(testloader.dataset),
    100. * correct / len(testloader.dataset)))

#Torchattacks

In [ ]:
!pip install torchattacks

In [ ]:
import torchattacks

In [ ]:
from torch.nn.modules import normalization
def test( model, device, test_loader, epsilon ):

    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in test_loader:

        # Send the data and label to the device
        data, target = data.to(device), target.to(device) #.to(device)
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue


        # Call Attack, you can change it to any other attack
        #attack = torchattacks.FFGSM(model, eps=epsilon, alpha=0.5) #8/255
        #attack = torchattacks.FGSM(model, eps=epsilon) #8/255
        #attack = torchattacks.DIFGSM(model, eps=epsilon, alpha=8/255)
        #attack = torchattacks.MIFGSM(model, eps=epsilon, alpha=8/255, steps=10, decay=1.0)
        #attack = torchattacks.SINIFGSM(model, eps=epsilon, alpha=8/255)
        #attack = torchattacks.TIFGSM(model, eps=epsilon, alpha=8/255)
        #attack = torchattacks.RFGSM(model, eps=epsilon, alpha=8/255)
        attack = torchattacks.PGD(model, eps=epsilon, alpha=8/255, steps=20, random_start=True)
        #attack = torchattacks.TPGD(model, eps=epsilon, alpha=8/255, steps=20)
        #attack = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
        #attack = torchattacks.attacks.cw.CW(model.cpu(), c=1, kappa=0, steps=50, lr=0.01)
        #data_denorm = denorm(data)
        attack.set_normalization_used(mean=[0.1307], std=[0.3081])
        #attack.save( test_loader, save_path=None, verbose=True, return_verbose=False,
        #     save_predictions=False, save_clean_inputs=False, save_type='float')
        #attack._normalization_applied=True
        #attack.normalization_used(mean=(0.1307,), std=(0.3081,))
        #normalize_data = attack.normalize(data)
        perturbed_data = attack(data.cpu(), target.cpu())

        #perturbed_data = inverse_normalize(data)#transforms.Normalize((0.1307,), (0.3081,))(perturbed_data)
        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == target.item():
            correct += 1

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [ ]:
epsilons = [0, .05, .1, .15, .2, .25, .3, .35, .4]
accuracies_torchattack = []
examples_torchattack = []
device = 'cuda'
# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(model.cuda(), device, testloader, eps)
    accuracies_torchattack.append(acc)
    examples_torchattack.append(ex)